In [2]:
import warnings
import scanpy as sc
import numpy as np
import pandas as pd
import time
import sys

import matplotlib.pyplot as plt
import seaborn as sns
import hdbscan
from scipy.stats import hypergeom, pearsonr

import matplotlib.cm as cm
import matplotlib.colors as mcolors 
import time
from statsmodels.stats.multitest import multipletests


sys.path.append('../3_DE_analysis/')
from DE_analysis_utils import *

pd.set_option('display.max_rows', 100)
sc.set_figure_params(figsize=(20, 4))

/Users/rzhu/miniconda3/envs/deseq/lib/python3.11/site-packages/scanpy/_utils/__init__.py:33: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  from anndata import __version__ as anndata_version
/Users/rzhu/miniconda3/envs/deseq/lib/python3.11/site-packages/scanpy/__init__.py:24: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  if Version(anndata.__version__) >= Version("0.11.0rc2"):
/Users/rzhu/miniconda3/envs/deseq/lib/python3.11/site-packages/scanpy/readwrite.py:16: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  if Version(anndata.__version__) >= Version("0.11.0rc2"):


In [3]:
cluster_nde75_ntotal50 = pd.read_csv('../../metadata/clustering_results.csv', index_col=0)
corr_df_all = pd.read_csv('../../../../3_expts/processed_data/analysis_largefiles/nde75ntotal50_gene_across_condition_correlation_matrix.csv', index_col=0)

In [4]:
cluster_name = []
corr = []
cluster_size = []
cluster_gene_size = []
cluster_member = []
rest_count = []
stim8hr_count = []
stim48hr_count = []
cluster_member_with_condition = []

for i, cl in enumerate(cluster_nde75_ntotal50['hdbscan'].unique()):
    df = cluster_nde75_ntotal50[cluster_nde75_ntotal50['hdbscan']==cl].copy()
    
    cluster_name.append(int(cl))
    cluster_size.append(len(df))
    cluster_gene_size.append(len(cluster_nde75_ntotal50[cluster_nde75_ntotal50['hdbscan']==cl].target_contrast_corrected.unique())) # number of unique regulator genes
    cluster_member.append(cluster_nde75_ntotal50[cluster_nde75_ntotal50['hdbscan']==cl].target_contrast_gene_name_corrected.unique().tolist()) # identity of regulator genes
    df_corr = corr_df_all.loc[cluster_nde75_ntotal50[cluster_nde75_ntotal50['hdbscan']==cl].index, cluster_nde75_ntotal50[cluster_nde75_ntotal50['hdbscan']==cl].index].copy()
    np.fill_diagonal(df_corr.values, 0)
    corr.append(np.mean(df_corr))

    list1 = df['target_contrast_gene_name_corrected'].tolist()
    list2 = df['culture_condition'].tolist()
    cluster_member_with_condition.append([f"{item1}_{item2}" for item1, item2 in zip(list1, list2)])

    rest_count.append(np.sum(df['culture_condition']=='Rest'))
    stim8hr_count.append(np.sum(df['culture_condition']=='Stim8hr'))
    stim48hr_count.append(np.sum(df['culture_condition']=='Stim48hr'))

cluster_df = pd.DataFrame({'cluster': cluster_name,
                           'intracluster_corr': corr,
                           'cluster_size': cluster_size,
                           'cluster_gene_size': cluster_gene_size,
                           'cluster_member': cluster_member,
                           'rest_count': rest_count,
                           'stim8hr_count': stim8hr_count,
                           'stim48hr_count': stim48hr_count,
                           'cluster_member_with_condition': cluster_member_with_condition})

### Pathway enrichment analysis

In [5]:
import gseapy
from gseapy import Msigdb
from gseapy import barplot, dotplot

msig = Msigdb()
kegg_gene_sets = msig.get_gmt(category= 'c2.cp.kegg_legacy', dbver="2025.1.Hs")
kegg_gene_sets = {key: value for key, value in kegg_gene_sets.items() if len(value) <= 200} # filter out terms that are too broadly defined
reactome_gene_sets = msig.get_gmt(category= 'c2.cp.reactome', dbver="2025.1.Hs")
reactome_gene_sets = {key: value for key, value in reactome_gene_sets.items() if len(value) <= 200} # filter out terms that are too broadly defined

In [7]:
corum_df = pd.read_csv('../../metadata/enrichment_database/corum_humanComplexes.txt', delimiter='\t', index_col='complex_id')
stringdb = pd.read_csv('../../metadata/enrichment_database/9606.clusters.proteins.v12.0.txt.gz', delimiter='\t', compression='gzip')
protein_info = pd.read_csv('../../metadata/enrichment_database/9606.protein.info.v12.0.txt.gz', delimiter='\t', compression='gzip')
cluster_info = pd.read_csv('../../metadata/enrichment_database/9606.clusters.info.v12.0.txt.gz', delimiter='\t', compression='gzip')
stringdb_df = pd.merge(stringdb, protein_info, left_on='protein_id', right_on='#string_protein_id')
stringdb_df = pd.merge(stringdb_df, cluster_info, left_on='cluster_id', right_on='cluster_id')
stringdb_df = stringdb_df[stringdb_df.cluster_size<=200].copy()

corum_complexes = {}
for _, row in corum_df.iterrows():
    complex_name = row['complex_name']
    subunits = set(row['subunits_gene_name'].split(';'))
    corum_complexes[complex_name] = subunits
corum_complexes = {key: value for key, value in corum_complexes.items() if len(value) <= 200} # filter out terms that are too broadly defined

stringdb_complexes = {}
for cluster_id in stringdb_df.cluster_id.unique():
    stringdb_complexes[cluster_id] = set(stringdb_df[stringdb_df.cluster_id==cluster_id].preferred_name)

In [8]:
def assess_complex_enrichment(df, complexes, cluster_label, gene_name_label):
    """
    Assess pathway/complex enrichment with Benjamini-Hochberg FDR correction.
    """
    de_genes_per_cluster = {}
    clusters = df[cluster_label].unique()
    for cluster in clusters:
        cluster_genes = df[df[cluster_label] == cluster][gene_name_label]
        de_genes_per_cluster[cluster] = set(cluster_genes)

    N = len(df[gene_name_label].unique())
    
    all_result = []
    
    # 1. Collect all raw p-values first
    for cluster, de_genes in de_genes_per_cluster.items():
        n = len(de_genes)
        
        for complex_name, subunits in complexes.items():
            K = len(subunits)
            overlap = de_genes.intersection(subunits)
            k = len(overlap)
            
            # Filter for non-trivial overlaps (at least 2 genes)
            if (n > 0 and K > 0) and (k > 1): 
                pval = hypergeom.sf(k - 1, N, K, n)
                
                current_result = {
                    'cluster': cluster,
                    'complex': complex_name,
                    'overlap_genes': list(overlap),
                    'overlap_fraction': len(overlap)/n,
                    'raw_p_value': pval,
                    'complex_size': K,
                    'overlap_size': len(overlap),
                    'cluster_size': n
                }
                all_result.append(current_result)
            elif (n > 0 and K > 0) and (k <= 1): 
                current_result = {
                    'cluster': cluster,
                    'complex': complex_name,
                    'overlap_genes': list(overlap),
                    'overlap_fraction': len(overlap)/n,
                    'raw_p_value': 1,
                    'complex_size': K,
                    'overlap_size': len(overlap),
                    'cluster_size': n
                }
                all_result.append(current_result)
    
    # 2. Create DataFrame
    all_result_df = pd.DataFrame(all_result)

    # 3. Apply FDR Correction (Benjamini-Hochberg)
    if not all_result_df.empty:
        # We correct across all tests performed for this specific dictionary of complexes
        # method='fdr_bh' is the standard Benjamini-Hochberg procedure
        rejects, fdr_pvals, _, _ = multipletests(all_result_df['raw_p_value'], alpha=0.05, method='fdr_bh')
        all_result_df['fdr'] = fdr_pvals
        
        # Sort by cluster and significance
        all_result_df = all_result_df.sort_values(by=['cluster', 'fdr', 'raw_p_value'])
        
        # 4. Extract best result per cluster
        # We simply take the top hit (lowest FDR) for each cluster
        best_result_df = all_result_df.drop_duplicates(subset=['cluster'], keep='first').copy()
        
    else:
        # Handle empty case
        best_result_df = pd.DataFrame(columns=['cluster', 'complex', 'overlap_genes', 'overlap_fraction', 
                                             'raw_p_value', 'fdr', 'complex_size', 'overlap_size', 'cluster_size'])

    return all_result_df, best_result_df

In [9]:
def run_enrichment_analysis(df, cluster_label, gene_name_label):
    # STRINGDB enrichment
    stringdb_enrichment_all, stringdb_enrichment_best = assess_complex_enrichment(df, stringdb_complexes, cluster_label, gene_name_label)
    stringdb_enrichment_all = pd.merge(stringdb_enrichment_all, stringdb_df[['cluster_id', 'best_described_by']].drop_duplicates(), left_on='complex', right_on='cluster_id')
    stringdb_enrichment_all = stringdb_enrichment_all.drop(columns=['cluster_id'])
    stringdb_enrichment_best = pd.merge(stringdb_enrichment_best, stringdb_df[['cluster_id', 'best_described_by']].drop_duplicates(), left_on='complex', right_on='cluster_id')
    stringdb_enrichment_best = stringdb_enrichment_best.drop(columns=['cluster_id'])
    # Corum enrichment
    corum_enrichment_all, corum_enrichment_best = assess_complex_enrichment(df, corum_complexes, cluster_label, gene_name_label)
    # KEGG enrichment
    kegg_enrichment_all, kegg_enrichment_best = assess_complex_enrichment(df, kegg_gene_sets, cluster_label, gene_name_label)
    # Reactome enrichment
    reactome_enrichment_all, reactome_enrichment_best = assess_complex_enrichment(df, reactome_gene_sets, cluster_label, gene_name_label)

    # Summarize results
    enrichment_df1 = pd.merge(corum_enrichment_best, stringdb_enrichment_best, on='cluster', how='outer', suffixes=('_corum', '_stringdb'))
    enrichment_df2 = pd.merge(kegg_enrichment_best, reactome_enrichment_best, on='cluster', how='outer', suffixes=('_kegg', '_reactome'))
    enrichment_df = pd.merge(enrichment_df1, enrichment_df2, on='cluster', how='outer')
    
    return enrichment_df, corum_enrichment_all, stringdb_enrichment_all, kegg_enrichment_all, reactome_enrichment_all

### Check regulator enrichment

In [10]:
enrichment_df,\
corum_enrichment_all,\
stringdb_enrichment_all,\
kegg_enrichment_all,\
reactome_enrichment_all = run_enrichment_analysis(cluster_nde75_ntotal50[['hdbscan', 'target_contrast_gene_name_corrected']], 'hdbscan', 'target_contrast_gene_name_corrected')

In [11]:
# Add cluster information
enrichment_df = pd.merge(cluster_df, enrichment_df, on='cluster', how='outer')

In [ ]:
enrichment_df.to_parquet('./results/clustering_nde75ntotal50_enrichment.parquet')
enrichment_df.to_csv('./results/clustering_nde75ntotal50_enrichment.csv')